# make sure you dont use the internet. 
this notbook run for me 10 hours... stay tune for updates

In [327]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import h5py
from io import BytesIO

In [328]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
from torchvision.models import vgg16

In [329]:
torch.manual_seed(1729)

In [330]:
device = ("cuda" if torch.cuda.is_available() 
          else "mps" if torch.backends.mps.is_available() 
          else "cpu")
torch.set_default_device(device)
print(f"device: {device}")

device: mps


# IMPORTANT !!!    remember to update MODEL_PATH !!!
* upload as a model in the right top corner (in kaggle)
* copy its path and paste it in MODEl_PATH in IN_KAGGLE

In [331]:
IN_KAGGLE = "KAGGLE_KERNEL_RUN_TYPE" in os.environ
IN_COLAB = "COLAB_GPU" in os.environ

if IN_COLAB:
    from google.colab import userdata    
    # only the first time
    # os.environ["KAGGLE_KEY"] = "d0ebb5786a5a5439881827924cf0ccbd"
    # os.environ["KAGGLE_USERNAME"] = "yuda03979"
    # 
    # ! kaggle competitions download isic-2024-challenge
    # ! unzip isic-2024-challenge.zip
    MODEL_PATH = "?"
    IMG_DIR = "/content/test-image.hdf5"
    CSV_TEST_PATH = "/content/test-metadata.csv"
elif IN_KAGGLE:
    MODEL_PATH = "_model_.pth"
    IMG_DIR = "/kaggle/input/isic-2024-challenge/test-image.hdf5"
    CSV_TEST_PATH = "/kaggle/input/isic-2024-challenge/test-metadata.csv"
else:
    MODEL_PATH = "_model_.pth"
    IMG_DIR = "/Users/yuda/Desktop/data_bases/isic-2024-challenge/test-image.hdf5"
    CSV_TEST_PATH = "/Users/yuda/PycharmProjects/my_isic2024/isic-2024-challenge/test-metadata.csv"

# create custom Dataset with reading hdf5

In [332]:
class ISICDataset(Dataset):
    
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        self.file = h5py.File(IMG_DIR,'r')

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        binary_data = self.file[self.df.loc[idx, "isic_id"]][()]
        img = Image.open(BytesIO(binary_data)).convert("RGB")
        
        if self.transform:
            img = self.transform(img)
        return img


# choose/define your model (its layers and functions)

In [333]:
# an example. you also can make a class or python-function
model = vgg16(weights=False) # internet cant be use
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 1)

/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


# upload model _model_.pth file and update the model weights

In [334]:
model = model.to(device)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

/var/folders/tw/t4_6l2cn5hz3hqm6tjzgwk5c0000gn/T/ipykernel_12329/624715535.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH))


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [335]:
BATCH_SIZE = 128
IMG_SIZE = (224, 224)

In [336]:
ISIC_df = pd.read_csv(CSV_TEST_PATH)
ISIC_df = ISIC_df[["isic_id"]]
ISIC_df

,isic_id
0,ISIC_0015657
1,ISIC_0015729
2,ISIC_0015740


In [337]:
transform = transforms.Compose([transforms.Resize(IMG_SIZE),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [338]:
test_dataset = ISICDataset(ISIC_df, transform)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False) # cant be shuffled

# the predictions on the model

In [339]:
pred_arr = np.array([])
def test(dataloader, model):
    model.eval()
    global pred_arr
    with torch.no_grad():
        for batch, imgs in enumerate(dataloader):
            imgs = imgs.to(device)

            pred_labels = model(imgs)
            pred_labels = torch.sigmoid(pred_labels)
            
            pred_arr = np.concatenate((pred_arr, pred_labels.cpu().flatten().detach().numpy()), axis=None)
            
            # print(f"{batch} from {len(dataloader)}") # => to see the progress. you can also do if batch % 50 == 0 then print...
    return pred_arr


In [340]:
ISIC_df["target"] = test(test_dataloader, model)
ISIC_df.head()

,isic_id,target
0,ISIC_0015657,0.949813
1,ISIC_0015729,0.005314
2,ISIC_0015740,0.004101


In [341]:
ISIC_df.to_csv("submission.csv", index=False)